In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from arch import arch_model
from arch.univariate import GARCH, EWMAVariance 
from sklearn import linear_model
import scipy.stats as stats
from statsmodels.regression.rolling import RollingOLS
from sklearn.linear_model import LinearRegression
import seaborn as sns
import warnings
from scipy.stats import norm
warnings.filterwarnings("ignore")
pd.set_option("display.precision", 5)

In [13]:
def tangency_portfolio(df, ann=12):
    N = df.shape[1]
    mu = df.mean() * ann #mean
    sig = df.cov() * ann #covariance
    sig_inv = np.linalg.inv(sig) #inverse of covariance
    scaling = np.ones(N)@sig_inv@mu
    om_t = (1/scaling) * sig_inv @ mu
    
    return pd.DataFrame(om_t, index=df.columns, columns=['weights_tangency'])

In [7]:
factors = pd.read_excel('midterm_2_data_pricing.xlsx', 1, index_col=0)
assets = pd.read_excel('midterm_2_data_pricing.xlsx', 2, index_col=0)

In [21]:
weights = tangency_portfolio(assets)
weights

,weights_tangency
NG1,0.05745
KC1,-0.07284
CC1,0.07447
LB1,0.08663
CT1,-0.00954
SB1,0.06361
LC1,0.12894
W1,-0.01044
S1,0.02731
C1,0.08483


In [15]:
def portfolio_stats(df, weights, ann=12):
    N = df.shape[1]
    mu = df.mean() * ann #mean
    sig = df.cov() * ann #covariance
    
    mu_p = weights.T @ mu
    sig_p = np.sqrt(weights.T @ sig @ weights)
    sharpe_p = mu_p/sig_p
    
    print('Mean: ', mu_p, ',\nVol: ', sig_p, ',\nSharpe Ratio: ', sharpe_p)

In [22]:

weights = weights.squeeze()
portfolio_stats(assets, weights)

Mean:  0.08751317697704902 ,
Vol:  0.11631378494806958 ,
Sharpe Ratio:  0.7523886959410777


In [19]:

### Minimum return, VaR, maximum drawdown, Value at risk

def tail_risk(df):
    tr_df = pd.DataFrame(data = None)
    tr_df['Min return'] = df.min()
    tr_df['VaR-5th'] = df.quantile(.05)
    cum_ret = (1 + df).cumprod()
    rolling_max = cum_ret.cummax()
    drawdown = (cum_ret - rolling_max) / rolling_max
    tr_df['Max Drawdown'] = drawdown.min()
    
    return tr_df

In [25]:
portfolio = assets @ weights
portfolio_df = pd.DataFrame(portfolio, columns=['Portfolio'])
tail_risk(portfolio_df)

,Min return,VaR-5th,Max Drawdown
Portfolio,-0.13799,-0.04139,-0.2533


In [28]:
mean_by_var = portfolio_df.mean() / portfolio_df.quantile(.05)
mean_by_var_each_asset = assets.mean() / assets.quantile(.05)
mean_by_var_each_asset.append(mean_by_var)

NG1         -0.06043
KC1         -0.03068
CC1         -0.05433
LB1         -0.07926
CT1         -0.03600
SB1         -0.05499
LC1         -0.02647
W1          -0.05023
S1          -0.04854
C1          -0.04956
GC1         -0.11230
SI1         -0.07162
HG1         -0.07898
PA1         -0.06487
Portfolio   -0.17620
dtype: float64

In [30]:
#assets up to 2017
assets_2017 = assets.loc[:'2017-12-31']
weights_2017 = tangency_portfolio(assets_2017)

#Use the data through 2017 to estimate the tangency weights.
# Apply these tangency weights to the data from 2018-2021.

assets_2018_2021 = assets.loc['2018-01-01':]

portfolio_2018_2021 = assets_2018_2021 @ weights_2017
weights_2017 = weights_2017.squeeze()
portfolio_stats(assets_2018_2021, weights_2017)

Mean:  0.0740842652549948 ,
Vol:  0.09893740684909451 ,
Sharpe Ratio:  0.7487993430835794


#### Consider the data for crude oil, (CL1), found in the same file, but on sheet “factors (excess returns)”. Suppose an investor wants to hold crude oil (CL1), but wants to hedge out the exposure to NG1 and KC1 (both series are found on the assets (excess returns) tab we’ve been using.)
- (5pts) Report the regression-based hedge ratio. Do NOT include an intercept in the regres- sion.
- (5pts) Report the mean, volatility, and Sharpe ratios of CL1 and the hedged version of CL1. Annualize the statistics.

In [32]:
def ts_test(df, factor_df, factors, constant = True,annualization=12):
    res = pd.DataFrame(data = None, index = df.columns, columns = ['alpha','f_1','f_2', 'r_2', 'treynor', 'info'])
    
    for port in df.columns:
        y = df[port]
        if constant:
            X = sm.add_constant(factor_df[factors])
        else:
            X = factor_df[factors]
        model = sm.OLS(y, X).fit()
        
        if constant:
            beta = model.params[1:]
            alpha = model.params[0] * annualization
            information_ratio = model.params[0] * np.sqrt(annualization) / model.resid.std()
        else:
            beta = model.params
    
        treynor = df[port].mean() * annualization / beta[0]
        tracking_error = model.resid.std() * np.sqrt(annualization)
        if constant:
            res.loc[port] = [alpha, model.params[1], model.params[2], model.rsquared, treynor, information_ratio]
        else:
            res.loc[port] = [None, model.params[0], model.params[1], model.rsquared, treynor, None]
    return res

#df is the asset portfolios which we are regressing
#factor_df is the factor data
#intercept is whether we want to include an intercept in the regression

In [46]:
crude_oil = factors['CL1']
crude_oil = pd.DataFrame(crude_oil, columns=['CL1'])

test = ts_test(crude_oil, assets, ['NG1','KC1'], constant = False)
test

,alpha,f_1,f_2,r_2,treynor,info
CL1,None,0.13103,0.10637,0.04114,0.82951,None


In [48]:
test.iloc[0,1]

0.1310324190100659

In [65]:
crude_oil

,CL1,NG1,KC1,CL1_hedge
Date,,,,
2000-01-31,0.07559,0.01820,-0.01294,0.07033
2000-02-29,0.09664,0.00431,-0.01118,0.10351
2000-03-31,-0.12070,0.00812,0.00355,-0.13237
2000-04-30,-0.04772,0.00812,-0.00911,-0.04674
2000-05-31,0.12204,0.05003,-0.00310,0.07511
...,...,...,...,...
2021-02-28,0.17816,0.01058,0.01203,0.15556
2021-03-31,-0.03805,-0.00771,-0.01038,-0.01996
2021-04-30,0.07471,0.01623,0.01417,0.04432


In [63]:
crude_oil['NG1'] = pd.DataFrame((test.iloc[0,1] * assets['NG1']),columns=['NG1'])
crude_oil['KC1'] = pd.DataFrame((test.iloc[0,2] * assets['KC1']),columns=['KC1'])
crude_oil['CL1_hedge'] = crude_oil['CL1'] - (crude_oil['NG1'] + crude_oil['KC1'])
crude_oil                 

,CL1,NG1,KC1,CL1_hedge
Date,,,,
2000-01-31,0.07559,0.01820,-0.01294,0.07033
2000-02-29,0.09664,0.00431,-0.01118,0.10351
2000-03-31,-0.12070,0.00812,0.00355,-0.13237
2000-04-30,-0.04772,0.00812,-0.00911,-0.04674
2000-05-31,0.12204,0.05003,-0.00310,0.07511
...,...,...,...,...
2021-02-28,0.17816,0.01058,0.01203,0.15556
2021-03-31,-0.03805,-0.00771,-0.01038,-0.01996
2021-04-30,0.07471,0.01623,0.01417,0.04432


In [50]:
#Calculate mean, standard deviation and sharpe ratio
def mean_vol_sharpe(df,ann=12):
    mean = df.mean() * ann
    volatility = df.std() * np.sqrt(ann)
    sharpe_ratio = mean/volatility
    return pd.DataFrame({'mean': mean, 'volatility': volatility, 'sharpe_ratio': sharpe_ratio})

In [64]:
mean_vol_sharpe(crude_oil)

,mean,volatility,sharpe_ratio
CL1,0.10869,0.39164,0.27753
NG1,0.01887,0.07050,0.26760
KC1,0.00478,0.03413,0.14010
CL1_hedge,0.08505,0.38395,0.22150


In [94]:
## Regress [SPY] or anything against a list of factors to estimate. Lagged Regression
## you can decide the weight
def lagged_reg(df, y_col, X_col, weight=100, lag=1, intercept = True, annual_fac=12):
    y = df[y_col]
    if intercept == True:
        X = sm.add_constant(df[X_col].shift(lag))
    else:
        X = df[X_col].shift(lag)
    
    model = sm.OLS(y, X, missing = 'drop').fit()
    reg_df = model.params.to_frame('Regression Parameters')
    reg_df.loc['r-squared'] = model.rsquared
    
    if intercept == True:
        reg_df.loc['const'] *= annual_fac
        final = reg_df.loc['const'][0]/12
    else:
        final = 0
    
    reg_df = reg_df.drop('const')
    reg_df = reg_df.drop('r-squared')
    
    for i in reg_df.index:
        final += reg_df.loc[i][0] * df[i]
    
    final = final * weight
    
    final_series = (final.shift() * df[y_col]).dropna()
    
    return model, final.shift().dropna()/weight, final_series

In [82]:
signal = pd.read_excel('final_exam_data.xlsx', index_col=0)
spy = pd.read_excel('final_exam_data.xlsx',1,index_col=0)
signal["SPY"] = spy["SPY"]
signal

,Level,Slope,Inflation Growth,SPY
date,,,,
1993-02-28,6.03,2.11,-0.29752,0.01067
1993-03-31,6.03,2.07,-0.63735,0.02241
1993-04-30,6.05,2.22,-0.65537,-0.02559
1993-05-31,6.16,1.92,-0.38643,0.02697
1993-06-30,5.80,1.77,-0.29241,0.00367
...,...,...,...,...
2021-06-30,1.45,1.20,0.54436,0.02247
2021-07-31,1.24,1.05,0.00730,0.02441
2021-08-31,1.30,1.10,0.10292,0.02976


In [95]:
reg, a, b = lagged_reg(signal, 'SPY', ['Level','Slope','Inflation Growth'], weight=100, lag=1, intercept = True, annual_fac=12)

In [98]:
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    SPY   R-squared:                       0.030
Model:                            OLS   Adj. R-squared:                  0.021
Method:                 Least Squares   F-statistic:                     3.462
Date:                Mon, 05 Dec 2022   Prob (F-statistic):             0.0166
Time:                        17:20:45   Log-Likelihood:                 606.42
No. Observations:                 344   AIC:                            -1205.
Df Residuals:                     340   BIC:                            -1189.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const                0.0238      0.007      3.224      0.001       0.009       0.038
Level               -0.0023      0.001     -1.702      0.090      -0.005       0.000
Slope               -0.0055      0.003     -1.906      0.057      -0.011       0.000
Inflation Growth    -0.0129      0.004     -2.928      0.004      -0.022      -0.004
==============================================================================
Omnibus:                       29.508   Durbin-Watson:                   1.987
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               40.703
Skew:                          -0.616   Prob(JB):                     1.45e-09
Kurtosis:                       4.150   Cond. No.                         15.7
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [99]:
a = a.to_frame('a')
a

,a
date,
1993-03-31,0.00205
1993-04-30,0.00665
1993-05-31,0.00601
1993-06-30,0.00394
1993-07-31,0.00439
...,...
2021-06-30,0.00513
2021-07-31,0.00679
2021-08-31,0.01503


In [100]:
b = b.to_frame('b')
b

,b
date,
1993-03-31,0.00459
1993-04-30,-0.01702
1993-05-31,0.01621
1993-06-30,0.00145
1993-07-31,-0.00213
...,...
2021-06-30,0.01154
2021-07-31,0.01657
2021-08-31,0.04473


In [108]:
spy['active'] = b['b']
spy

,SPY,active
date,,
1993-02-28,0.01067,NaN
1993-03-31,0.02241,0.00459
1993-04-30,-0.02559,-0.01702
1993-05-31,0.02697,0.01621
1993-06-30,0.00367,0.00145
...,...,...
2021-06-30,0.02247,0.01154
2021-07-31,0.02441,0.01657
2021-08-31,0.02976,0.04473


In [109]:
mean_vol_sharpe(spy)

,mean,volatility,sharpe_ratio
SPY,0.11144,0.14598,0.76341
active,0.16656,0.17716,0.94019


In [103]:
## Var, skewness, kurtosis, expected shortfall, maximum drawdown
def risk_stats(data, q=0.05):
    df = data.copy()
    df.index = data.index.date
    report = pd.DataFrame(columns = df.columns)
    
    report.loc['Skewness'] = df.skew()
    report.loc['Excess Kurtosis'] = df.kurtosis()
    report.loc['VaR (negated)'] = -df.quantile(q)
    report.loc['Expected Shortfall (negated)'] = -df[df < df.quantile(q)].mean()
    
    cum_ret = (1 + df).cumprod()
    rolling_max = cum_ret.cummax()
    drawdown = (cum_ret - rolling_max) / rolling_max
    report.loc['Max Drawdown'] = drawdown.min()
    report.loc['MDD Start'] = None
    report.loc['MDD End'] = drawdown.idxmin()
    report.loc['Recovery Date'] = None
    
    for col in df.columns:
        report.loc['MDD Start', col] = (rolling_max.loc[:report.loc['MDD End', col]])[col].idxmax()
        recovery_df = (drawdown.loc[report.loc['MDD End', col]:])[col]
        
        try:
            report.loc['Recovery Date', col] = recovery_df[recovery_df >= 0].index[0]

        except:
            report.loc['Recovery Date', col] = None
            report.loc['Recovery period (days)'] = None
    report.loc['Recovery period (days)'] = (report.loc['Recovery Date'] - report.loc['MDD Start']).dt.days
    return round(report,4)

#risk_stats(ltcm_ex).iloc[:3,1:4].T

In [110]:
risk_stats(spy)

,SPY,active
Skewness,-0.61941,0.62533
Excess Kurtosis,1.26508,5.6021
VaR (negated),0.06938,0.06479
Expected Shortfall (negated),0.09406,0.10112
Max Drawdown,-0.50798,-0.28331
MDD Start,2007-10-31,2007-10-31
MDD End,2009-02-28,2009-02-28
Recovery Date,2012-03-31,2009-08-31
Recovery period (days),1613,670
